## How Operator Payout Calculated (skip to bottom to see reward calculation)

This notebook calculates the ETH earned by each node operator from 19990000-20860000 or from May 31 to Sept 30th. The methodology for determining how much each node operator earned involves calculating the rewards accumulated by all the node contracts and eigenpods between block from_block=19990000 and block to_block=20860000. Additionally, all rewards claimed within this time period are also included in the calculation.

The final formula is:
payout_amount = (balance_at_to_block - balance_at_from_block + all_claimed_rewards)

**Rewards still in the consensus layer, which have not been partially withdrawn to the withdrawal safe, are not part of the payout amount, as they will be paid out in the next quarter.**

Eigenpods: Contracts where consensus rewards are deposited.
Node Contracts: Where MEV rewards are deposited.

## Import Libraries

In [1]:
import aiohttp
import asyncio
from dotenv import load_dotenv
import json
import os
import time
from web3 import Web3
from web3.eth import Contract, AsyncEth

## Constants/Environment Variables

In [2]:
#env variables
load_dotenv()
WEB3_URL=os.getenv("WEB3_URL")
BEACONCHAIN_API_KEY=os.getenv("BEACONCHAIN_API_KEY")
BEACON_URL=os.getenv("BEACON_URL")
GRAPH_URL=os.getenv("MAINNET_GRAPH_URL")
ETHERSCAN_KEY=os.getenv("ETHERSCAN_KEY")
DISCORD_WEBHOOK=str(os.getenv("DISCORD_WEBHOOK"))

#contract addresses
LIQUIDITY_POOL_ADDRESS= "0x308861A430be4cce5502d0A12724771Fc6DaF216"
P2P_ADDRESS= "0x70eb1ef5626c3D7598a5730F3bF7a328ed8F7FC5"  
SSV_ADDRESS= "0x27c229FE370C1195c45ff1953e96acd741aA48c0"
OBOL_ADDRESS="0xbeFa010044579f47C94E230C14C62811ff8ed1fc"
MAKERDAO_MULTICALL_ADDRESS = "0xeefba1e63905ef1d7acba5a8513c70307c1ce441"
MAKERDAO_MULTICALL_ADDRESS = "0xeefba1e63905ef1d7acba5a8513c70307c1ce441"
ETHERFIVIEW_CONTRACT = "0x2ecd155405cA52a5ca0e552981fF44A8252FAb81"

#provider
provider = Web3(Web3.HTTPProvider(WEB3_URL))
asyncProvider = Web3(Web3.AsyncHTTPProvider(WEB3_URL), modules={'eth': (AsyncEth,)})

#ABIs
MAKERDAO_MULTICALL_ABI = json.load(open("./abis/multicall.json"))
ETHERFIVIEW_CONTRACT_ABI = json.load(open("./abis/etherfiviewer.json"))
st_contract_abi = json.load(open("./abis/eeth.json"))
lido_abi = json.load(open("./abis/lido.json"))

BLOCK_NUMBER=None
TIMESTAMP=None
SLOT_NUMBER=None
BATCH_SIZE=50
BATCH_SIZE=50
PAGINATION_LIMIT=1000
VALIDATORS_LEN = 1000 #rename this
MULTICALL_BATCH = 1000

def compute_slot_at_timestamp(timestamp):
    genesisTime = 1606824023
    return (timestamp - genesisTime) // 12

## Declare Classes

In [3]:

class Multicall:
    def __init__(self, w3: Web3, chain='mainnet', custom_address=None, custom_abi=None):
        address = Web3.to_checksum_address(MAKERDAO_MULTICALL_ADDRESS)
        abi = MAKERDAO_MULTICALL_ABI
        self.multicall = w3.eth.contract(address=address, abi=abi)

    async def call(self, calls: list,blocknumber: any = 'latest') -> list:
        res = self.multicall.functions.aggregate(calls).call(block_identifier=blocknumber)
        return res

    def create_call(self, contract: Contract, fn_name: str,args: list) -> tuple:
        return (contract.address, contract.encodeABI(fn_name=fn_name, args=args))


In [4]:

class Subgraph():

    def __init__(self, network: str):
        self.network = network
        self.validators = []
        self.blockno = BLOCK_NUMBER
        self.config_map = {
            "mainnet": {
                "graph_url": GRAPH_URL,
            }
        }
        self.queries = {
            "validators": '''
            query GetValidators($limit: Int!, $value: String!) {
              validators(
                first: $limit
                block: {number: %s}
                        where: {
          and: [{ id_gt: $value }, { blockNumber_gt: "18620000" }, {phase_in: [LIVE, EXITED, FULLY_WITHDRAWN, WAITING_FOR_APPROVAL, BEING_SLASHED]}]
        }
              ) {
                id
                phase
                validatorPubKey
                BNFTHolder
                TNFTHolder
                etherfiNode
                bid {
                    bidderAddress
                }
              }
            }
            ''',
        }
        

    async def query_graph(self, query: str, field_name: str, variables: dict[str, any]):
        self.blockno = BLOCK_NUMBER
        GRAPH_URL = self.config_map[self.network]["graph_url"]
        async with aiohttp.ClientSession() as session:
            async with session.post(GRAPH_URL, json={'query': query, 'variables': variables}) as response:
                data = await response.json()
                if 'data' in data and field_name in data['data']:
                    return data['data'][field_name]
                else:
                    raise Exception(f"Error fetching data from subgraph: {data}")
      
    async def fetch_large_data_from_graph(self, query: str, field_name: str):   
        try:
            async def get_batch(value):
                try:
                    variables = {'limit': PAGINATION_LIMIT, 'value': value}
                    res = await self.query_graph(query, field_name, variables)
                    return res
                except Exception as error:
                    raise Exception(str(error))
            res = []
            value = " "
            while True:
                batch = await get_batch(value)
                res.extend(batch)
                if len(batch) < PAGINATION_LIMIT:
                    break
                
                value = batch[-1]["id"]
                time.sleep(0.1)
            print("length of large data is: ", len(res))
            return res
        except Exception as error:
            raise Exception("Error: query_from_graph: {}".format(error))
        
    async def fetch_validators(self):
        query = self.queries["validators"]
        validators = await self.fetch_large_data_from_graph(query, "validators")
        return validators
    


Note: Using Beaconchain API to determine address Validator Consensus Rewards Go (Eigenpod)

In [5]:

class BeaconchainRateLimiter:
  def __init__(self, limit, interval):
    self.limit = limit
    self.interval = interval
    self.semaphore = asyncio.Semaphore(limit)

  async def getSempahore(self):
    await self.semaphore.acquire()
    asyncio.get_event_loop().call_later(self.interval, self.release)

  def release(self):
    self.semaphore.release()

class BeaconchainAPI():
    __instance = None
    __inited = False
    def __new__(cls):
        if BeaconchainAPI.__instance is None:
            BeaconchainAPI.__instance = object.__new__(cls)
        return BeaconchainAPI.__instance

    def __init__(self):
      self.beacon_api = "https://beaconcha.in/api/v1"
      if BeaconchainAPI.__inited:
        return
      BeaconchainAPI.__inited = True
      self.rate_limiter = BeaconchainRateLimiter(10, 3) # 10 requests per 1.3 seconds

    async def call(self, url):
      await self.rate_limiter.getSempahore()
      try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    if "data" in data:
                        return data["data"]
                else:
                    print("Error: ", response.status)
                    raise Exception("Beaconchain.in returns status " + str(response.status))
      except aiohttp.ClientError as e:
        print("Error: ", e)

    async def fetch_batched_val_beacon_info(self, indicesOrPubkey, batch_size, network: str) -> list:
        all_info = [self.get_group_val_beacon_info(indicesOrPubkey[i: i+batch_size], network) for i in range(0, len(indicesOrPubkey), batch_size)]
        all_info = await asyncio.gather(*all_info)
        retval = [info for group_info in all_info for info in group_info]
        return retval

    async def get_group_val_beacon_info(self, indicesOrPubkey: list[str], network: str) -> list:
        try:
          api = self.beacon_api
          url = f"{api}/validator/{','.join(indicesOrPubkey)}?apikey={BEACONCHAIN_API_KEY}"
          vals_info = await self.call(url)
          # There is an edge case that if there is only one validator index, the api will return an object instead of array
          if not isinstance(vals_info, list):
            print(f"Finished fetching validator info! Validators fetched: {1}")
            return [vals_info]
          else:
            return vals_info
        except Exception as error:
          raise Exception(str(error))
        
    async def get_beacon_info_by_pubkeys(self, pubkeys):
      return await self.fetch_batched_val_beacon_info(pubkeys, BATCH_SIZE, "mainnet")
    
    async def get_timestamp_by_block(self, block_number):
      url = f"{self.beacon_api}/execution/block/{block_number}?apikey={BEACONCHAIN_API_KEY}"
      return (await self.call(url))[0]['timestamp']
      

## Class Instances and Contracts

In [6]:
#contracts
etherfiview_contract = provider.eth.contract(address=Web3.to_checksum_address(ETHERFIVIEW_CONTRACT), abi=ETHERFIVIEW_CONTRACT_ABI)
multicall_contract = provider.eth.contract(address=Web3.to_checksum_address(MAKERDAO_MULTICALL_ADDRESS), abi=MAKERDAO_MULTICALL_ABI)

#objects
subgraph = Subgraph("mainnet")
multicall = Multicall(provider)
beacon_api = BeaconchainAPI()

## Library of Functions

In [7]:

async def get_block_number_by_timestamp(timestamp, network: str = "mainnet"):
  async with aiohttp.ClientSession() as session:
    async with session.get(f"https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey={ETHERSCAN_KEY}") as response:
      if response.status == 200:
        data = await response.json()
        return int(data["result"])
      else:
        raise Exception("Error fetching block number from Etherscan")
      
async def get_balance(address, block_number=None, rpc_url=WEB3_URL):
    return provider.eth.get_balance(address, block_identifier=block_number)
      

In [8]:
async def get_balances(addresses): 
    balances = []
    calls = []
    counter = 0
    for i in range(0, len(addresses), MULTICALL_BATCH):
        batch = addresses[i:min(i+MULTICALL_BATCH, len(addresses))]
        calls = [multicall.create_call(multicall_contract, "getEthBalance", [Web3.to_checksum_address(addy)]) for addy in batch]
        results = await multicall.call(calls, blocknumber=BLOCK_NUMBER)
        for j, response in enumerate(results[1]):
            counter+=1
            balances.append(int(response.hex(), 16))
    addressBalances = dict(zip(addresses, balances))
    return addressBalances


## Etherfi Protocol Details Functions

In [9]:
def get_num_vals_shared_contract(validators):
    val_contract_count = {}
    for val in validators:
        if val["etherfiNode"] in val_contract_count:
            val_contract_count[val["etherfiNode"]] += 1
        else:
            val_contract_count[val["etherfiNode"]] = 1
    return val_contract_count

async def get_pubkeys(validators):
    pubkeys = []
    for val in validators:
        if(val["validatorPubKey"]):
            pubkeys.append(val["validatorPubKey"])
    return pubkeys

async def get_etherfi_node_contracts(validators): #execution rewards go here
    execution_contracts = []
    for val in validators:
        if(val["etherfiNode"]):
            execution_contracts.append(val["etherfiNode"])
    return list(set(execution_contracts))

async def get_eigenpods(beacon_info): #consensus rewards go here
    eigenpods = ['0x' + info['withdrawalcredentials'][26:]for info in beacon_info]
    return list(set(eigenpods))

def get_pubkey_to_eigenpods(pubkeys, beacon_info):
    pubkey_to_eigenpods = {}
    for val in beacon_info:
        pubkey_to_eigenpods[(val["pubkey"]).lower()] = "0x" + val["withdrawalcredentials"][26:]
    return pubkey_to_eigenpods

In [10]:
async def get_delayed_withdrawal_eth(ids):
    delayedWithdrawalBalance = []
    wsEth = []
    eigenPod = []
    for i in range(0, len(ids), VALIDATORS_LEN):
        batch = ids[i:min(i+VALIDATORS_LEN, len(ids))]
        res = etherfiview_contract.functions.EtherFiNodesManager_splitBalanceInExecutionLayer(batch).call(block_identifier=BLOCK_NUMBER)
        wsEth += res[0]
        eigenPod += res[1]
        delayedWithdrawalBalance += res[2]
    return wsEth, eigenPod, delayedWithdrawalBalance

async def get_ids_etherfinodes(validators):
    ids = []
    etherfi_contracts = []
    for val in validators:
        if(val["etherfiNode"] not in etherfi_contracts):
            ids.append(int(val["id"], 16))
            etherfi_contracts.append(val["etherfiNode"])
    return ids, etherfi_contracts


## Variables

### Modify BlockNo and Timestamp Below

In [11]:
##### PROVIDE BLOCK NUMBER  ######
BLOCK_NUMBER = 19990000
#################################

BLOCK_NUMBER = (await asyncProvider.eth.get_block_number())-40 if BLOCK_NUMBER is None else BLOCK_NUMBER

In [12]:
#differentiate different types of validators
def filter_validators_by_phase(validators, phases):
    filtered_validators = []
    for val in validators:
        if val["phase"] in phases:
            filtered_validators.append(val)
    return filtered_validators

## Node Operator Rewards Calculation

In [13]:
#assume balance of live or exited is 32eth
async def get_rewards_payout_to_no(validators, pubkey_to_eigenpod, beacon_balances, eigenpods_balances, etherfi_node_balances, delayed_withdrawal_eth_balance, vals_per_contract):
    node_op_payout = {}
    for val in validators:
        no = val["bid"]["bidderAddress"]
        num_validators_sharing_the_safe = vals_per_contract[val["etherfiNode"]]
        eigenpod = pubkey_to_eigenpod[(val["validatorPubKey"]).lower()]
        beacon_balance = 0
        delayed_withdrawal_balance = delayed_withdrawal_eth_balance[val["etherfiNode"]]
        eigenpod_balance =  eigenpods_balances[eigenpod]
        etherfi_node_balance = etherfi_node_balances[val["etherfiNode"]]
        total_balance = beacon_balance + (delayed_withdrawal_balance + eigenpod_balance + etherfi_node_balance) / num_validators_sharing_the_safe / 10**18
        if(total_balance < 31):
            total_balance += 32
        assert(val["TNFTHolder"].lower() == LIQUIDITY_POOL_ADDRESS.lower())
        acrruedRewards = total_balance - 32
        node_op_payout[no] = node_op_payout.get(no, 0) + 0.05 * acrruedRewards
    return node_op_payout

async def get_rewards_payout_for_withdrawn_validator_no(validators, pubkey_to_eigenpod, eigenpods_balances, etherfi_node_balances, vals_per_contract):
    node_op_payout = {}
    for val in validators:
        no = val["bid"]["bidderAddress"]
        num_validators_sharing_the_safe = vals_per_contract[val["etherfiNode"]]
        eigenpod = pubkey_to_eigenpod[(val["validatorPubKey"]).lower()]
        eigenpod_balance =  eigenpods_balances[eigenpod]
        etherfi_node_balance = (etherfi_node_balances[val["etherfiNode"]])
        accruedRewards = (eigenpod_balance + etherfi_node_balance) / num_validators_sharing_the_safe / 10**18
        node_op_payout[no] = node_op_payout.get(no, 0) + 0.05 * accruedRewards
    return node_op_payout

In [14]:
FROM_BLOCK = 19990000 #May-31-2024
TO_BLOCK = 20860000 #Sep-30-2024

In [15]:
#Get total rewards at START_BLOCK grouped by node operator
no_to_payout_end = {}

#takes 3-7mins to run
#validators with 1 eth deposited to beacon deposit contract
BLOCK_NUMBER = TO_BLOCK
print("Block number: ", BLOCK_NUMBER)


all_validators_at_end = await subgraph.fetch_large_data_from_graph((subgraph.queries["validators"]) % BLOCK_NUMBER, "validators")
print("length of all validators at start: ", len(all_validators_at_end))

# # validators with 32 eth
max_eth_validators = filter_validators_by_phase(all_validators_at_end, ["LIVE", "EXITED", "BEING_SLASHED"])
print("length of max eth validators: ", len(max_eth_validators))
max_eth_validators_ids, _ = await get_ids_etherfinodes(max_eth_validators)

max_eth_pubkeys = await get_pubkeys(max_eth_validators)
max_eth_validators_beacon_info = await beacon_api.get_beacon_info_by_pubkeys(max_eth_pubkeys)
max_eth_validators_etherfinodes = await get_etherfi_node_contracts(max_eth_validators)
max_eth_validators_eigenpods = await get_eigenpods(max_eth_validators_beacon_info)

#validators with 0 eth towards validators but has rewards in eigenpods
pod_rewards_only_validators = filter_validators_by_phase(all_validators_at_end, ["FULLY_WITHDRAWN"])
pod_rewards_only_pubkeys = await get_pubkeys(pod_rewards_only_validators)
pod_rewards_only_validators_beacon_info = await beacon_api.get_beacon_info_by_pubkeys(pod_rewards_only_pubkeys)
pod_rewards_only_validators_etherfinodes = await get_etherfi_node_contracts(pod_rewards_only_validators)
pod_rewards_only_validators_eigenpods = await get_eigenpods(pod_rewards_only_validators_beacon_info)

# validators with 32 eth
max_eth_validator_beacon_balances = []
max_eth_validator_eigenpods_balances = await get_balances(max_eth_validators_eigenpods)
max_eth_validators_etherfinode_balances = await get_balances(max_eth_validators_etherfinodes)
_, _, max_eth_validator_delayed_balance = await get_delayed_withdrawal_eth(max_eth_validators_ids)
delayed_withdrawal_eth_balance = dict(zip(max_eth_validators_etherfinodes, max_eth_validator_delayed_balance))


#validators with 0 eth towards validators but has rewards in eigenpods
pod_rewards_only_validators_eigenpods_balances = await get_balances(pod_rewards_only_validators_eigenpods)
pod_rewards_only_validators_etherfinode_balances = await get_balances(pod_rewards_only_validators_etherfinodes)

vals_per_contract = get_num_vals_shared_contract(max_eth_validators + pod_rewards_only_validators)
max_validator_external_payout = await get_rewards_payout_to_no(max_eth_validators, get_pubkey_to_eigenpods(max_eth_pubkeys, max_eth_validators_beacon_info), max_eth_validator_beacon_balances, max_eth_validator_eigenpods_balances, max_eth_validators_etherfinode_balances, delayed_withdrawal_eth_balance, vals_per_contract)
pod_rewards_only_external_payout = await get_rewards_payout_for_withdrawn_validator_no(pod_rewards_only_validators, get_pubkey_to_eigenpods(pod_rewards_only_pubkeys, pod_rewards_only_validators_beacon_info), pod_rewards_only_validators_eigenpods_balances, pod_rewards_only_validators_etherfinode_balances, vals_per_contract)


for no, balance in max_validator_external_payout.items():
    no_to_payout_end[no] = (balance + pod_rewards_only_external_payout.get(no, 0))



Block number:  20860000
length of large data is:  57484
length of all validators at start:  57484
length of max eth validators:  54305


### All Node Operator Rewards claimed by the protocol earned within START_BLOCK to END_BLOCK

In [16]:
#get all withdrawals
no_to_withdrawals = {}
node_to_no = {}

node_manager_abi = json.load(open("./abis/etherfinodesmanager.json"))
NODE_MANAGER_ADDRESS = '0x8b71140ad2e5d1e7018d2a7f8a288bd3cd38916f'

node_manager_contract = asyncProvider.eth.contract(address=Web3.to_checksum_address(NODE_MANAGER_ADDRESS), abi=node_manager_abi)

logs = []
for i in range(FROM_BLOCK, TO_BLOCK, 10000):
    logs += (await node_manager_contract.events.PartialWithdrawal().get_logs(fromBlock=i, toBlock=i+10000))
    logs += (await node_manager_contract.events.FullWithdrawal().get_logs(fromBlock=i, toBlock=i+10000))

for val in max_eth_validators + pod_rewards_only_validators:
    node_to_no[(val["etherfiNode"]).lower()] = val["bid"]["bidderAddress"]

print("length of logs: ", len(logs))

for log in logs:
    no = node_to_no[(log["args"]["etherFiNode"]).lower()] 
    toTreasury = (log["args"]["toTreasury"]) / 2 / 10**18
    no_to_withdrawals[no] = no_to_withdrawals.get(no, 0) + toTreasury
    
print(no_to_withdrawals)

length of logs:  13498
{'0x7c0576343975a1360ceb91238e7b7985b8d71bf4': 21.636757819731805, '0xd624feff4b4e77486b544c93a30794ca4b3f10a2': 18.425556655546995, '0x78ca32ac90d7f99225a3b9288d561e0cb3744899': 19.379796923096414, '0xf92204022cdf7ee0763ef794f69427a9dd9a7834': 2.7009978882721293, '0xafbd66706f90bc56d29c39a260930b34b2757ed8': 11.55602988401831, '0x83b55df61cd1181f019df8e93d46bafd31806d50': 11.171181963198238, '0xc24fd5c977b982e2eb1ca1ae3f2d2daf28ace35a': 20.20190134086999, '0xb8db44e12eacc48f7c2224a248c8990289556fae': 14.253041976310826, '0x6916487f0c4553b9ee2f401847b6c58341b76991': 8.410344772173252, '0x771a428b865a3984085ed49a0ca550dd222bb60b': 5.394297405234651, '0x1876eccb4edd3ed95051c64824430fc7f1c8763c': 0.3481652708218696, '0x2b17924d0d2f3d70aefb07602c7926827677ba19': 0.03472647415, '0xd1208cc82765aa4dc696117d26f37388b6dcb6d5': 0.06250864655, '0xe2f8cefcdee51f48e3ce5c4deea3095c43369b36': 0.00852386259619853}


###  Calculate Total Rewards Accrued To Node Operator by START_BLOCK by querying all of a node operator's contract hold consensus and execution rewards. 

In [17]:
#Get total rewards at START_BLOCK grouped by node operator
no_to_payout_start = {}

#takes 3-7mins to run
#validators with 1 eth deposited to beacon deposit contract
BLOCK_NUMBER = FROM_BLOCK

all_validators_at_start = await subgraph.fetch_large_data_from_graph((subgraph.queries["validators"]) % FROM_BLOCK, "validators")

# # validators with 32 eth
max_eth_validators_start = filter_validators_by_phase(all_validators_at_start, ["LIVE", "EXITED", "BEING_SLASHED"])
max_eth_validators_ids_start, _ = await get_ids_etherfinodes(max_eth_validators_start)
max_eth_pubkeys_start = await get_pubkeys(max_eth_validators_start)
max_eth_validators_beacon_info_start = await beacon_api.get_beacon_info_by_pubkeys(max_eth_pubkeys_start)
max_eth_validators_etherfinodes_start = await get_etherfi_node_contracts(max_eth_validators_start)
max_eth_validators_eigenpods_start = await get_eigenpods(max_eth_validators_beacon_info_start)

#validators with 0 eth towards validators but has rewards in eigenpods
pod_rewards_only_validators_start = filter_validators_by_phase(all_validators_at_start, ["FULLY_WITHDRAWN"])
pod_rewards_only_pubkeys_start = await get_pubkeys(pod_rewards_only_validators_start)
pod_rewards_only_validators_beacon_info_start = await beacon_api.get_beacon_info_by_pubkeys(pod_rewards_only_pubkeys_start)
pod_rewards_only_validators_etherfinodes_start = await get_etherfi_node_contracts(pod_rewards_only_validators_start)
pod_rewards_only_validators_eigenpods_start = await get_eigenpods(pod_rewards_only_validators_beacon_info_start)


# validators with 32 eth
max_eth_validator_beacon_balances_start = []
max_eth_validator_eigenpods_balances_start = await get_balances(max_eth_validators_eigenpods_start)
max_eth_validators_etherfinode_balances_start = await get_balances(max_eth_validators_etherfinodes_start)
_, _, max_eth_validator_delayed_balance_start = await get_delayed_withdrawal_eth(max_eth_validators_ids_start)
delayed_withdrawal_eth_balance_start = dict(zip(max_eth_validators_etherfinodes_start, max_eth_validator_delayed_balance_start))


#validators with 0 eth towards validators but has rewards in eigenpods
pod_rewards_only_validators_eigenpods_balances_start = await get_balances(pod_rewards_only_validators_eigenpods_start)
pod_rewards_only_validators_etherfinode_balances_start = await get_balances(pod_rewards_only_validators_etherfinodes_start)

vals_per_contract_start = get_num_vals_shared_contract(max_eth_validators_start + pod_rewards_only_validators_start)
max_validator_external_payout_start = await get_rewards_payout_to_no(max_eth_validators_start, get_pubkey_to_eigenpods(max_eth_pubkeys_start, max_eth_validators_beacon_info_start), max_eth_validator_beacon_balances_start, max_eth_validator_eigenpods_balances_start, max_eth_validators_etherfinode_balances_start, delayed_withdrawal_eth_balance_start, vals_per_contract_start)
pod_rewards_only_external_payout_start = await get_rewards_payout_for_withdrawn_validator_no(pod_rewards_only_validators_start, get_pubkey_to_eigenpods(pod_rewards_only_pubkeys_start, pod_rewards_only_validators_beacon_info_start), pod_rewards_only_validators_eigenpods_balances_start, pod_rewards_only_validators_etherfinode_balances_start, vals_per_contract_start)


for no, balance in max_validator_external_payout_start.items():
    no_to_payout_start[no] = (balance + pod_rewards_only_external_payout_start.get(no, 0))

length of large data is:  41598


In [18]:

final_payment_amount = {}
for no, balance in no_to_payout_end.items():
    final_payment_amount[no] = balance + no_to_withdrawals.get(no, 0) - no_to_payout_start.get(no, 0)

In [19]:
#import csv and create mapping from no to name
import csv 
address_to_name = {}
with open("mainnet.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        address_to_name[row[0]] = row[1]
print(address_to_name)

{'0x1876eccb4edd3ed95051c64824430fc7f1c8763c': 'solo_staker', '0x2b17924d0d2f3d70aefb07602c7926827677ba19': 'p2p', '0x5bea039db7a516a2f1c258fe002c9ec9ac75730d': 'p2p ssv', '0x6916487f0c4553b9ee2f401847b6c58341b76991': 'nodemonster', '0x74b44741efbc3edc7f0cdade165eab1afe9c65ca': 'allnode-2', '0x771a428b865a3984085ed49a0ca550dd222bb60b': 'nethermind', '0x78ca32ac90d7f99225a3b9288d561e0cb3744899': 'allnodes', '0x7c0576343975a1360ceb91238e7b7985b8d71bf4': 'dsrv', '0x83b55df61cd1181f019df8e93d46bafd31806d50': 'A41', '0xafbd66706f90bc56d29c39a260930b34b2757ed8': 'chainnodes', '0xb8db44e12eacc48f7c2224a248c8990289556fae': 'finoa', '0xc24fd5c977b982e2eb1ca1ae3f2d2daf28ace35a': 'pier_two', '0xc9000c7c13e3498158d9cc1f6058aaffddd95b0a': 'blockdaemon', '0xd1208cc82765aa4dc696117d26f37388b6dcb6d5': 'ssv_dvt', '0xd624feff4b4e77486b544c93a30794ca4b3f10a2': 'cosmostation', '0xe2f8cefcdee51f48e3ce5c4deea3095c43369b36': 'infstones', '0xf92204022cdf7ee0763ef794f69427a9dd9a7834': 'validation_cloud', '0xfa

### Total Rewards Earned is:  Total Rewards Earned Upto Date - Rewards Earned last pay period + all claimed rewards this quarter

In [20]:
name_to_balance = {}
for address, name in address_to_name.items():
    amount = final_payment_amount[address]
    name_to_balance[name] = amount
    
name_to_balance

{'solo_staker': 1.895066822820157,
 'p2p': 7.29716897760002,
 'p2p ssv': 27.51743584955088,
 'nodemonster': 54.49951806402713,
 'allnode-2': 2.8426395222287364,
 'nethermind': 72.15456492300467,
 'allnodes': 102.22416690728141,
 'dsrv': 115.20959288485024,
 'A41': 63.464058864406795,
 'chainnodes': 74.80130177140046,
 'finoa': 47.68414164955007,
 'pier_two': 88.30073614837198,
 'blockdaemon': 8.939954893266856,
 'ssv_dvt': 49.48538165920212,
 'cosmostation': 74.70013580472803,
 'infstones': 12.827986404692867,
 'validation_cloud': 69.45208399578051,
 'Obol DVT': 12.271511982700066}